In [ ]:
import os
import numpy as np
# import scipy as 
import pandas as pd
import pandapower as pp
import math

Functions

In [ ]:
def ReadData(folder_name):
	"""
	Return the network data as a dictionary: {key=name, data=pandas dataframe}

	:param kind: folder of the different files
	:type kind: str
	:raise lumache.InvalidKindError: If the kind is invalid.
	:return: dictionary with the data
	:rtype: dict

	"""
	data = {}
	extensions = ['txt', 'csv']
	folder_path = os.path.join('../Data',folder_name)
	# iterate through all file
	for file in os.listdir(folder_path):
		#File expected as "name_file.file_extension"
		file_name, file_extension = file.split('.') 
		if file_extension in extensions:
			file_path = os.path.join(folder_path, file)
			file_data = pd.read_csv(file_path)
			data[file_name] = file_data
	return data

def ReadTimeseries(path_timeseries_data):
	"""
	Return a 

	:param kind: Optional "kind" of ingredients.
	:type kind: list[str] or None
	:raise lumache.InvalidKindError: If the kind is invalid.
	:return: The ingredients list.
	:rtype: list[str]

	"""
	data = ''
	return data

def BuildNetwork(network_data):
	"""
	Return a 

	:param kind: Optional "kind" of ingredients.
	:type kind: list[str] or None
	:raise lumache.InvalidKindError: If the kind is invalid.
	:return: The ingredients list.
	:rtype: list[str]

	"""

	net = pp.create_empty_network()
	for elements,data in network_data.items():
		if(elements=='buses'):
			choosable = []
			for b in data.itertuples(index=False):
				id_bus = pp.create_bus(net, name = b[0], vn_kv = b[1], type = b[2])
				choosable.append(id_bus%2) #TODO: find a more meaningful way to find the choosable buses
			net.bus['choosable'] = choosable
		elif(elements=='lines'):
			for l in data.itertuples(index=False):
				pp.create_line(net, name = l[0], from_bus = l[1], to_bus = l[2], length_km = l[3], std_type = l[4])
		elif(elements=='transformers'):
			for t in data.itertuples(index=False):
				pp.create_transformer_from_parameters(net, hv_bus=t[0], lv_bus=t[1], i0_percent=t[2], pfe_kw=t[3],
										vkr_percent=t[4], sn_mva=t[5], vn_lv_kv=t[6], vn_hv_kv=t[7], vk_percent=t[8])
		elif(elements=='loads'):
			for l in data.itertuples(index=False):
				pp.create_load(net, l[0], p_mw = l[1], q_mvar = l[2], name = l[3])
	pp.create_ext_grid(net,0)
	return net


In [ ]:
#Scenarios

#Nice example for PV hosting capacity
#https://github.com/e2nIEE/pandapower/blob/develop/tutorials/hosting_capacity.ipynb
def Chose_buses(net, percentuage, use_weights):
	#TODO: can be more complex: you may want to exclude some buses (ext-grid bus, ...), attach a PV only where load is, ...
	choosable_buses = net.bus[ net.bus['choosable'] == 1 ]
	elements_to_select = np.min([percentuage, len(choosable_buses)]) if percentuage>=1 else round(len(choosable_buses)*percentuage) #fixed number (1, 4, ...) or percentuages of the total
	if(use_weights == False):
		result = np.random.choice(net.bus.index, elements_to_select, replace=False)
	else:
		p = choosable_buses[use_weights]
		p = p / np.sum(p) #Make the sum of p equal to 1 (Required by numpy)
		result = np.random.choice(choosable_buses.index, elements_to_select, replace=False, p=p)
	return result

def PVscenario(net, p_mw, percentuage, use_weights=False):
	new_net = net.deepcopy() #TODO Deepcopy solve the problems but it may lead to memory leaks. [old]This function return the original element (net) modified not a new element. Be careful

	ids = Chose_buses(new_net, percentuage, use_weights)

	for i in ids:
		pp.create_sgen(new_net, i, p_mw=p_mw, q_mvar=0)
	return new_net


In [ ]:
#Network Validation
def IssueDetection(net):
	issues = {} #Dictionary of issues -> k: position (bus, line, ...), list of issues (OV, UV, ...)

	#TODO: change default values.
	#Over Voltage
	vm_pu_max = 1.05
	ll_max = 45

	at_least_one_issue = False

	#OVER VOLTAGE
	ov_issues = net.res_bus[ net.res_bus.vm_pu > vm_pu_max ]
	for i in ov_issues.iterrows():
			id = i[0]
			element = i[1]
			issues[f'Node {id}'] = [['OV', element.vm_pu]]
			at_least_one_issue = True

	#LINE LOADINGS
	ll_issues = net.res_line[ net.res_line.loading_percent > ll_max ]
	for i in ll_issues.iterrows():
			id = i[0]
			element = i[1]			
			# v = ['LoL', element.loading_percent]
			# if(id in issues.keys()): #line and node may have the same id number (semi issue #0)
			# 	issues[id].append(v)
			# else:
			# 	issues[id] = [v]
			issues[f'Line {id}'] = [['LoL', element.loading_percent]] #This solves the 'semi issue #0'. It is even more informative
			at_least_one_issue = True

	return issues, at_least_one_issue

In [ ]:
def CalculatePossibleCombinations(net, num_changes):
    choosable_elements = len(net.bus.choosable == True)
    n_combinations = math.comb(choosable_elements, num_changes) # comb(n,m) = n! / [(n-m)!m!], n>m
    return n_combinations

Build Network

In [ ]:
#Read network data
path_network_data = 'Test'
network_data = ReadData(path_network_data)

In [ ]:
#Read timeseries
path_timeseries_data = ''
timeseries = ReadTimeseries(path_timeseries_data) #TODO: define timeseries
timeseries = [1,2] #temp values

In [ ]:
net = BuildNetwork(network_data)

In [ ]:
#Tmp network
import pandapower.networks
from pandapower.plotting.plotly import simple_plotly
from pandapower.plotting.plotly import pf_res_plotly

# net = pp.networks.mv_oberrhein()
net = pp.networks.create_cigre_network_lv()
_ = simple_plotly(net)
net.bus['choosable'] = 1

# pp.runpp(net)
# _ = pf_res_plotly(net)

In [ ]:
#Find distance between buses and ext grid/s
import pandapower.topology as top

distances = top.calc_distance_to_bus(net, 0)
net.bus['distance (km)'] = distances
#Normalize between [d_min, d_max]
b_min = np.min(distances) #minimum of the measurements
b_max = np.max(distances) #maximum of the measurements
d_min = 0.0 #minimum of the wanted range 
d_max = 0.7 #maximum of the wanted range
net.bus['distance (norm)'] = (distances - b_min) / (b_max - b_min) * (d_max - d_min) + d_min

Simulation

In [ ]:
#Main

#initialize variable to be used
scenarios_type = ['PV', 'EV', 'HP', 'EVERYTHING'] #TODO: add others cases
scenario = 'PV'

num_changes = range(1,30) #x* in Amina's report. Can be a list like [0%, 10%, ...] but PV, EV or ...?
n_simulation_per_scenario = 10 #TODO: it can be a list [50, 1000, ...]
timeseries = ['t1'] #temp values

In [ ]:
import time
print(f'Num power flow calculations: {len(num_changes) * (n_simulation_per_scenario) * len(timeseries)}')
slow = False
wait_secs = 1
results = {} #TODO: can be a more complex stucture (class, ...)
#Dimention: [tech_change, n_simulation, time, loc, type]
scenario = 'PV'

for x in num_changes:
	result = {}

	n_combinations = CalculatePossibleCombinations(net, x)
	N = min(n_simulation_per_scenario, n_combinations)
	# print(f'X={x}. Simulationg for N={N} scenarios. (possible combinations: {n_combinations})')
	for n in range(N):

		issues = {}
		#Generate scenario - choose one possible scenario (s) out of the scenario space (S)
		if(scenario == scenarios_type[0]):
			new_net = PVscenario(net, p_mw=20/1000, percentuage=(x+1), use_weights = 'distance (norm)')
		else:
			_ = 'blabla' #TODO: to add the different cases implementation

		

		for t in timeseries:
			#TODO: Apply values for loads and sgen to the net
			pp.runpp(new_net)
			
			temp_issues, at_least_one_issue = IssueDetection(new_net)

			#1. Store only True/False
			# issues[t] = at_least_one_issue
			#2. Store all the issues or False if no issue is present
			issues[t] = temp_issues if at_least_one_issue==True else False
			#3. Store everithing
			issues[t] = new_net.res_bus.vm_pu
		
		result[n] = issues

	if slow:
		_ = pf_res_plotly(new_net, figsize=0.5)
		time.sleep(wait_secs)


	results[x] = result

In [ ]:
issues

In [ ]:
results

In [ ]:
def ResultsAnalyser(results):
	total_locations = 0
	total_problems = 0
	capacities = results.items()
	for capacity, scenarios in capacities:
		print(f'### Results for capacity {capacity}: ###')
		for scenario, timesteps in scenarios.items():
			print(f'\tScenario: {scenario}')
			for timestep, locations in timesteps.items():
				print(f'\t\tTimestep: {timestep}')
				if(locations!=False):
					print(f'\t\t\tProblems in {len(locations)} locations. IDs: {sorted(set(locations))}')
					total_locations += len(locations)

					type_issues = set()
					for issues in locations.values():
						[type_issues.add(i[0]) for i in issues]
						total_problems += len(issues)
					print(f'\t\t\tType(s) of issue(s): {type_issues}')
				else: print('\t\t\tNo issues')

		print()

	print(f"Total number of 'problematic' locations: {total_locations}")
	print(f"Total number of problems: {total_problems}")
ResultsAnalyser(results)

In [ ]:
import seaborn as sns

data = {}
capacities = results.items()
for capacity, scenarios in capacities:
    data[capacity] = []
    d = []
    for scenario, timesteps in scenarios.items():
        for timestep, voltages in timesteps.items():
            d.extend(voltages)
    data[capacity] = d

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
df = pd.DataFrame(data)
df = df.drop([0,1]) #Remove Trafo nad Ext_grid

for i in df.columns:
    fig.add_trace(go.Violin(y=df[i], name=i, points=False ))
    fig.add_trace(go.Box(y=df[i], name=i, boxpoints=False))
    fig.add_trace(go.Scatter(x=[i], y=[np.max(df[i])], showlegend=False ))
fig.add_hline(y=1.05)

fig.update_layout(
    title="",
    xaxis_title="Capacity (Number of PV added)",
    yaxis_title="Voltage level [p.u.]",
    legend_title=""
)

fig.show()

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")

In [ ]:
df

In [ ]:
df = pd.DataFrame(data)
d

In [ ]:
list(df.columns)

In [ ]:
list(np.max(df))

In [ ]:
_ = pf_res_plotly(new_net)

In [ ]:
pf_res_plotly(net)